In [1]:
import torch
import gc

def clear_gpu_memory():
    # Empty the cache
    torch.cuda.empty_cache()
    
    # Clear any leftover tensors
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if obj.is_cuda:
                    del obj
        except Exception:
            pass
    
    # Run garbage collector
    gc.collect()
    
    # Force CUDA to synchronize
    torch.cuda.synchronize()

# Example usage:
def check_memory():
    print(f"Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    print(f"Cached: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

check_memory()
clear_gpu_memory()
check_memory()

Allocated: 0.00 MB
Cached: 0.00 MB


/vol/bitbucket/lst20/lex-eval/lib/python3.12/site-packages/torch/distributed/distributed_c10d.py:359: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


Allocated: 0.00 MB
Cached: 0.00 MB


In [2]:
from model.engine import GemmaAdapter
from adapters.SemanticAdapter import SemanticAdapter

In [3]:
modelId ="google/gemma-2-9b-it"
model = GemmaAdapter(modelId)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
from adapters.rag import RAGAgent
rag = RAGAgent(model)


In [17]:
root_prompt = 'Which physicist developed the theory of general relativity, fundamentally altering our understanding of gravity?'
wiki_data = rag.retrieve_wiki_data_2(root_prompt)
contriever_closest_matches = rag.find_top3_contriever_matches(
    wiki_data=wiki_data, prompt=root_prompt
)
contriever_closest_matches

/vol/bitbucket/lst20/lex-eval/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /vol/bitbucket/lst20/lex-eval/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


[({'title': 'Introduction to general relativity',
   'content': 'General relativity is a theory of gravitation developed by Albert Einstein between 1907 and 1915. The theory of general relativity says that the observed gravitational effect between masses results from their warping of spacetime.\nBy the beginning of the 20th century, Newton\'s law of universal gravitation had been accepted for more than two hundred years as a valid description of the gravitational force between masses. In Newton\'s model, gravity is the result of an attractive force between massive objects. Although even Newton was troubled by the unknown nature of that force, the basic framework was extremely successful at describing motion.\nExperiments and observations show that Einstein\'s description of gravitation accounts for several effects that are unexplained by Newton\'s law, such as minute anomalies in the orbits of Mercury and other planets. General relativity also predicts novel effects of gravity, such as

In [18]:
contriever_response = rag.format_topk_wiki_answer(
    root_prompt,
    contriever_closest_matches
)

In [20]:
semantic_adapter = SemanticAdapter(model)
answer = semantic_adapter.wiki_rag_completions('a', contriever_response, root_prompt)
print(answer)

Albert Einstein 

